In [2]:
'''
Importing relevant libraries for project.
'''
import pandas as pd
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC

In [37]:
'''
Importing CSV file of all valid Australian postcodes as a list.
'''
POA_2016 = open('POA_2016_aust.csv')
valid_postcodes = csv.reader(POA_2016)

for row in valid_postcodes:
    valid_postcodes = row

In [38]:
valid_postcodes

['800',
 '810',
 '812',
 '815',
 '820',
 '822',
 '828',
 '829',
 '830',
 '832',
 '834',
 '835',
 '836',
 '837',
 '838',
 '839',
 '840',
 '841',
 '845',
 '846',
 '847',
 '850',
 '852',
 '853',
 '854',
 '860',
 '862',
 '870',
 '872',
 '873',
 '874',
 '875',
 '880',
 '885',
 '886',
 '2000',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024',
 '2025',
 '2026',
 '2027',
 '2028',
 '2029',
 '2030',
 '2031',
 '2032',
 '2033',
 '2034',
 '2035',
 '2036',
 '2037',
 '2038',
 '2039',
 '2040',
 '2041',
 '2042',
 '2043',
 '2044',
 '2045',
 '2046',
 '2047',
 '2048',
 '2049',
 '2050',
 '2052',
 '2060',
 '2061',
 '2062',
 '2063',
 '2064',
 '2065',
 '2066',
 '2067',
 '2068',
 '2069',
 '2070',
 '2071',
 '2072',
 '2073',
 '2074',
 '2075',
 '2076',
 '2077',
 '2079',
 '2080',
 '2081',
 '2082',
 '2083',
 '2084',
 '2085',
 '2086',
 '2087',
 '2088',
 '2089',
 '2090',
 '2092',
 '2093',
 '2094',
 '2095',
 '2096',
 '2097',


In [13]:
'''
Creating a counter for number of iterations and an empty master dictionary for scraped IGA store data to be appended to later on.
'''
Aus_postcodes = ['3204', '3141', '3000']
store_counter = 0
IGA_stores = { }

'''
Selenium set up and navigation to IGA 'Store locator' website.
'''
browser = webdriver.Chrome()
browser.get('https://www.iga.com.au/stores/#view=storelocator')

'''
The following block of code is for Selenium to simulate a user navigating through the IGA website and inputting postcodes into the search box.
Selenium will:

      Navigate to the search box, clear if there is any text in there, input a postcode, wait for the suggestions box to load, press the down arrow
      and then return to select the first suggestion of the dropdown list.
      
This code is within a for loop, which is iterating through the list of valid Australian postcodes.
'''
for postcode in Aus_postcodes:
    
    time.sleep(5)
    
    try:
        search_box = browser.find_element_by_id("sf-location-search")
    
    except:
        print("Cannot find search box")
    
    search_box.clear()
    search_box.send_keys(postcode)
    search_box.click()
    
    time.sleep(5)

    search_box.send_keys(Keys.ARROW_DOWN)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(5)
    
    """
    This next block of code is using Selenium to scrape the store name and store address for the search results loaded from the website. The results
    are being stored in a list for storage.
    """
    store_name = [ ]
    store_address = [ ]

    for store in browser.find_elements_by_xpath('.//span[@class = "sf-storename"]'):
        store_name.append(store.text)

    for address in browser.find_elements_by_xpath('.//p[@class = "sf-storeaddress"]'):
        store_address.append(address.text)

    '''
    Some basic text cleaning of the store address list.
    '''
    cleaned_store_address_list = [ ]

    for store in store_address:
        cleaned_store_address_list.append(store.replace('\n' , ','))
    
    """
    We are now iterating through the two lists of store name and store address and adding them to the master dictionary that we created earlier.
    """
    for i in range(len(store_name)):
        IGA_stores.update( {store_name[i] : cleaned_store_address_list[i] } )
    
    print(f"Stores for postcode: {postcode} have been added to the master dictionary")
    
    store_counter += 1
    
    #This isn't quite working!
    percentage_progress = float(store_counter)/len(Aus_postcodes)*100 
    print("Progress: ",round(percentage_progress,1),"%")
    
    """
    Now that all search results for this postcode have been extracted and stored, find and click on the change location button on the website, ready for 
    the next postcodes to be input into the search box.
    """
    change_location = browser.find_element_by_id("sf-location-change")
    change_location.click()

Stores for postcode: 3204 have been added to the master dictionary
Progress:  33.33 %
Stores for postcode: 3141 have been added to the master dictionary
Progress:  66.67 %
Stores for postcode: 3000 have been added to the master dictionary
Progress:  100.0 %


In [41]:
'''
Finally, take the fully populated master dictionary and output to a .csv file.
'''
result = pd.DataFrame(final_IGA_stores_list)
result.to_csv("scraped_IGA_stores.csv", index=False)